# Intermediate Loop Setup and Optimization

Testing intermediate loop setup with scipy optimize

## Method 1: Optimization of loop temperatures
Uses scipy.optimize to determine loop temperatures for intermediate and secondary loop for SC-CO2 cycle.

In [71]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.optimize as spo
import read_inputs, compute_cycle_efficiency
from compute_required_area_v2 import _compute_velocity
import compute_required_area_v2 as compute_required_area

from th_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Method 1.1: Optimization of hot leg temperature and loop dT
4 input variables are used:
1. Intermediate Loop Hot Leg Temperature
2. Intermediate Loop (Hot-Cold) Leg dT
3. Secondary Loop Hot Leg Temperature
4. Secondary Loop (Hot-Cold) Leg dT


In [69]:
""" Assume x to have the following structure
         0    1       2    3
x: [T2_hot, dT2, T3_hot, dT3]
"""
inputs = read_inputs.read_inputs()
primary_hot = inputs["Primary Hot Temperature (C)"]
primary_cold = inputs["Primary Cold Temperature (C)"]
primary_dT = primary_hot - primary_cold
# yapf:disable
cons = ({'type': 'ineq', 'fun': lambda x: x[3] - x[1]},
        {'type': 'ineq', 'fun': lambda x: primary_cold - (x[0]-x[1])}, # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: (x[0]-x[1]) - (x[2]-x[3])},  # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[0] - x[2]},                # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: 350000 -hx_calc(x, return_cost=True)}
       )
bounds = spo.Bounds(
    #         T2_hot,   dT2,       T3_hot,   dT3  Cost
    lb=[primary_cold, 150.0, primary_cold, 150.0], # Lower Bound
    ub=[ primary_hot, 200.0,  primary_hot, 200.0], # Upper Bound
    keep_feasible=True
)

# yapf:enable
def hx_calc(x, print_res=False, return_cost=False):
    t2h, dT2, t3h, dT3 = x
    t2c = t2h - dT2
    t3c = t3h - dT3
    inputs = read_inputs.read_inputs()
    inputs['Intermediate Hot Temperature (C)'] = t2h
    inputs['Intermediate Cold Temperature (C)'] = t2c
    inputs['Secondary Hot Temperature (C)'] = t3h
    inputs['Secondary Cold Temperature (C)'] = t3c
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results["Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000          # $/m3
        cost = HX_volume*132000*inflation # $
        LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
    except:
        return 1e5
    if print_res:
        print(f"Cost Fns: {LMTD}, {(5e-3*cost**(2/3))}")
        print(f"Temperatures: {t2h}, {t2c}, {t3h}, {t3c}")
        print(f"dTs         : {t2h-t2c}, {t3h-t3c}")
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(pd.Series(compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    if return_cost:
        return cost
    return LMTD + 5e-3*cost**(2/3)

# res = spo.minimize(hx_calc, [530, 530 - 155, 500, 500 - 180], bounds=bounds, constraints=cons)

init_guess = [primary_hot - 20, 165, primary_hot - 50, 190]
res = spo.minimize(hx_calc, init_guess, bounds=bounds, constraints=cons)
hx_calc(res.x, True)

/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LMTD2 = (deltaT_2h-deltaT_2c)/np.log(deltaT_2h/deltaT_2c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid value encountered in scalar divide
  LM

Cost Fns: 78.17832869358521, 24.61927438158612
Temperatures: 527.1421160578259, 365.8503957698766, 477.33166831191835, 316.0399480239692
dTs         : 161.29172028794926, 161.29172028794915
Vol:   [0.58274667 1.03298382]
Cost:  345507.80805387575
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)           477.331668
Secondary Cold Temperature (C)          316.039948
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Plate thickness (m)                          0.001
Channel Diameter (m)                       0.00135
Plate material                               SS316
Primary Pressure Drop (kPa)            

102.79760307517134

### Method 1.2: Optimization of hot and cold leg temperature 
4 input variables are used:
1. Intermediate Loop Hot Leg Temperature
2. Intermediate Loop Cold Leg Temperature
3. Secondary Loop Hot Leg Temperature
4. Secondary Loop Cold Leg Temperature


In [70]:
""" Assume x to have the following structure
         0        1       2        3   4   5
x: [T2_hot, T2_cold, T3_hot, T3_cold, L1, L2]
"""
inputs = read_inputs.read_inputs()
primary_hot = inputs["Primary Hot Temperature (C)"]
primary_cold = inputs["Primary Cold Temperature (C)"]

#yapf:disable
cons = ({'type': 'ineq', 'fun': lambda x: (x[0] - x[1]) - (primary_hot-primary_cold)}, # dT1 > dT2
        {'type': 'ineq', 'fun': lambda x: (x[2] - x[3]) - (x[0] - x[1])},                # dT2 > dT3
        {'type': 'ineq', 'fun': lambda x: primary_cold - x[1]}, # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[1] - x[3]},         # Cold Leg T gap
        {'type': 'ineq', 'fun': lambda x: primary_hot - x[0]},  # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: x[0] - x[2]},         # Hot Leg T gap
        {'type': 'ineq', 'fun': lambda x: 350000 -hx_calc(x, return_cost=True)}
       )
bounds = spo.Bounds(
    #         T2_hot,      T2_cold,        T3_hot,     T3_cold  Ch.Diam  dP1  dP2
    lb=[primary_cold,          200, primary_cold,          200], # Lower Bound
    ub=[ primary_hot, primary_cold,  primary_hot, primary_cold], # Upper Bound
)

# yapf:enable
def hx_calc(x, print_res=False, return_cost=False):
    t2h, t2c, t3h, t3c = x
    inputs = read_inputs.read_inputs()
    inputs['Intermediate Hot Temperature (C)'] = t2h
    inputs['Intermediate Cold Temperature (C)'] = t2c
    inputs['Secondary Hot Temperature (C)'] = t3h
    inputs['Secondary Cold Temperature (C)'] = t3c
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results["Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000          # $/m3
        cost = HX_volume*132000*inflation # $
        LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
    except:
        return 1e7
    if print_res:
        print(f"Cost Fns: {LMTD}, {(cost**(2/3)*5e-3)}")
        print(f"Temperatures: {t2h}, {t2c}, {t3h}, {t3c}")
        print(f"dTs         : {t2h-t2c}, {t3h-t3c}")
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(pd.Series(compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    if return_cost:
        return cost
    return LMTD + 5e-3*cost**(2/3)

res = spo.minimize(hx_calc, [530, 530 - 165, 500, 500 - 190], bounds=bounds, constraints=cons)
# res = spo.minimize(hx_calc, [545, 545 - 155, 525, 525 - 180], bounds=bounds, constraints=cons)
hx_calc(res.x, True)

/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/tmp/ipykernel_54540/283989095.py:41: RuntimeWarning: invalid value encountered in scalar divide
  LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
/tmp/ipykernel_54540/283989095.py:41: RuntimeWarning: invalid value encountered in scalar divide
  LMTD = ((primary_hot-t3h) - (primary_cold-t3c))/(np.log((primary_hot-t3h)/(primary_cold-t3c)))
/home/user/phd/sc_co2/compute_required_area_test.py:168: RuntimeWarning: invalid value encountered in scalar divide
  LMTD1 = (deltaT_1h-deltaT_1c)/np.log(deltaT_1h/deltaT_1c)
/home/user/phd/sc_co2/compute_required_area_test.py:172: RuntimeWarning: invalid val

Cost Fns: 76.80000000000008, 24.855995301447408
Temperatures: 524.0269052604001, 374.02690526040004, 472.45205167905635, 322.4520516790565
dTs         : 150.00000000000006, 149.99999999999983
Vol:   [0.62414886 1.01494108]
Cost:  350502.99291649996
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)           472.452052
Secondary Cold Temperature (C)          322.452052
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Plate thickness (m)                          0.001
Channel Diameter (m)                       0.00135
Plate material                               SS316
Primary Pressure Drop (kPa)          

101.65599530144749

## Method 2: Optimization of loop parameters
Uses scipy.optimize to determine optimal HX parameters given fixed loop temperatures (primary, intermediate, secondary)

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.optimize as spo
import read_inputs, compute_cycle_efficiency
from compute_required_area import _compute_velocity
import compute_required_area as compute_required_area

from th_functions import *

In [125]:
def cost_calc(x, print_res=False):
    # Assume x to have the following structure
    Tloss_i, dT_i, d_channel, dP_p, dP_s = x
    inputs = read_inputs.read_inputs()
    inputs["Intermediate Tloss"] = Tloss_i
    inputs["Intermediate dTMultiplier"] = dT_i
    inputs["Channel Diameter (m)"] = d_channel
    inputs["Primary Pressure Drop (kPa)"] = dP_p
    inputs["Secondary Pressure Drop (kPa)"] = dP_s
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results["Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000          # $/m3
        cost = HX_volume*132000*inflation # $
    except:
        return 8e5
    if print_res:
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(pd.Series(compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    return cost

res = spo.minimize(cost_calc, [15.3, 1.127, 1.354e-3, 104.2, 200], bounds=bounds)
print(res)
cost_calc(res.x, print_res=True)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 397619.4634388773
        x: [ 1.626e+01  1.107e+00  1.345e-03  1.042e+02  1.500e+02]
      nit: 19
      jac: [-2.602e+00 -2.790e+01 -2.209e+03  0.000e+00 -4.083e+02]
     nfev: 216
     njev: 36
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
Vol:   [0.70017   1.1592551]
Cost:  397619.4634388773
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)                  500
Secondary Cold Temperature (C)                 300
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Intermediate Tloss               

397619.4634388773